<a href="https://colab.research.google.com/github/NigarSultana156/499A/blob/main/objectDetectionRCNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub

In [ ]:
# Paths for videos and dataset
video_paths = [
    '/content/drive/MyDrive/top-view-multi-person-tracking-2020/cam1_1hour.mp4',
    '/content/drive/MyDrive/top-view-multi-person-tracking-2020/cam2_1hour.mp4',
    '/content/drive/MyDrive/top-view-multi-person-tracking-2020/cam3_1hour.mp4',
    '/content/drive/MyDrive/top-view-multi-person-tracking-2020/cam5_1hour.mp4',
    '/content/drive/MyDrive/top-view-multi-person-tracking-2020/cam6_1hour.mp4'
]

# Extract ZIP file for labeled images
zip_file_path = '/content/drive/MyDrive/top-view-multi-person-tracking-2020/labeled_images/train.zip'
extraction_path = '/content/extracted_files'
os.makedirs(extraction_path, exist_ok=True)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

test_zip_file_path = '/content/drive/MyDrive/top-view-multi-person-tracking-2020/labeled_images/test.zip'
test_extraction_path = '/content/extracted_files'
os.makedirs(test_extraction_path, exist_ok=True)
with zipfile.ZipFile(test_zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(test_extraction_path)

print("Files extracted to:", extraction_path, "and", test_extraction_path)


Files extracted to: /content/extracted_files and /content/extracted_files


In [ ]:
# Load Faster R-CNN model from TensorFlow Hub
model = hub.load("https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1")

In [ ]:
def detect_objects(frame):
    # Convert the frame to a tensor
    input_tensor = tf.convert_to_tensor([frame], dtype=tf.uint8)

    # Run detection
    detections = model(input_tensor)

    # Process the detection results
    boxes = detections["detection_boxes"].numpy()[0]
    scores = detections["detection_scores"].numpy()[0]
    classes = detections["detection_classes"].numpy()[0]

    # Filter detections with a high confidence score and only keep persons (class 1)
    detection_threshold = 0.5
    filtered_boxes = boxes[(scores >= detection_threshold) & (classes == 1)]
    filtered_scores = scores[(scores >= detection_threshold) & (classes == 1)]

    # Draw bounding boxes on the frame and add counts and confidence scores
    for i, box in enumerate(filtered_boxes):
        ymin, xmin, ymax, xmax = box
        start_point = (int(xmin * frame.shape[1]), int(ymin * frame.shape[0]))
        end_point = (int(xmax * frame.shape[1]), int(ymax * frame.shape[0]))
        frame = cv2.rectangle(frame, start_point, end_point, (0, 255, 0), 3)  # Green box, thickness 3

        # Add count and confidence score as text
        count_text = str(i + 1)
        confidence_text = f"{filtered_scores[i]:.2f}"
        cv2.putText(frame, count_text, (start_point[0], start_point[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.putText(frame, confidence_text, (start_point[0], start_point[1] - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame


In [ ]:
# Initialize video captures
captures = [cv2.VideoCapture(video_path) for video_path in video_paths]
for capture in captures:
    assert capture.isOpened()

im_width = int(captures[0].get(cv2.CAP_PROP_FRAME_WIDTH))
im_height = int(captures[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
out_size = (im_width * 3, im_height * 2)

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('/content/output_with_detection.avi', fourcc, 20, out_size)


In [ ]:
num_frames = 500  # Set a lower number of frames to process
frame_skip = 2    # Skip every 2 frames to reduce processing frequency

current_frame = 0

while current_frame < num_frames:
    frames = []
    for capture in captures:
        ret, frame = capture.read()
        if not ret:  # Exit if the frame read was not successful
            frames.append(None)
        else:
            frames.append(frame)

    if any(frame is None for frame in frames):
        break  # Exit if any video ends

    # Apply detection on each frame
    frames = [detect_objects(frame) for frame in frames]

    shape1 = list(frames[0].shape)
    shape1[1] = 160
    shape2 = list(frames[0].shape)
    shape2[1] = 1280 - 160

    upper = np.hstack((np.zeros(shape1), frames[3], frames[4], np.zeros(shape2)))
    lower = np.hstack((frames[0], frames[1], frames[2]))

    merged_frame = np.vstack((upper, lower))
    out.write(np.uint8(merged_frame))

    if current_frame % 100 == 0:
        print("Processed frame %d of %d" % (current_frame, num_frames))

    current_frame += frame_skip  # Increment by frame_skip to process fewer frames

out.release()


Processed frame 0 of 500
Processed frame 100 of 500
Processed frame 200 of 500


In [ ]:
import matplotlib.pyplot as plt

# Example data
frame_indices = list(range(1, 501))  # Frame indices from 1 to 500
detection_counts = [np.random.randint(0, 5) for _ in frame_indices]  # Random detection counts for demonstration

plt.figure(figsize=(12, 6))
plt.plot(frame_indices, detection_counts, marker='o', linestyle='-', color='b')
plt.title('Number of Person Detections Over Time')
plt.xlabel('Frame Number')
plt.ylabel('Number of Detections')
plt.grid()
plt.show()


In [ ]:
import seaborn as sns

# Example confidence scores (random for demonstration)
confidence_scores = [np.random.rand() for _ in range(1000)]  # Generate 1000 random confidence scores

plt.figure(figsize=(12, 6))
sns.histplot(confidence_scores, bins=20, kde=True)
plt.title('Distribution of Confidence Scores for Person Detections')
plt.xlabel('Confidence Score')
plt.ylabel('Frequency')
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve

# Example precision and recall data (random for demonstration)
y_true = np.random.randint(0, 2, size=1000)  # Random binary true labels
y_scores = np.random.rand(1000)  # Random scores

precision, recall, _ = precision_recall_curve(y_true, y_scores)

plt.figure(figsize=(12, 6))
plt.plot(recall, precision, marker='o', linestyle='-', color='g')
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.grid()
plt.show()


In [ ]:
# Function to visualize detected bounding boxes on an image
def visualize_detections(image, boxes, scores, classes):
    for i, box in enumerate(boxes):
        ymin, xmin, ymax, xmax = box
        start_point = (int(xmin * image.shape[1]), int(ymin * image.shape[0]))
        end_point = (int(xmax * image.shape[1]), int(ymax * image.shape[0]))
        cv2.rectangle(image, start_point, end_point, (0, 255, 0), 3)  # Green box
        cv2.putText(image, f'{scores[i]:.2f}', (start_point[0], start_point[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    return image

# Example of visualizing the first detected frame
detected_frame = visualize_detections(cam1, filtered_boxes, filtered_scores, filtered_classes)  # Assuming these variables exist
plt.imshow(cv2.cvtColor(detected_frame, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Detected Persons in Frame')
plt.show()


In [ ]:
# Convert to MP4 format
!ffmpeg -i /content/output_with_detection.avi -vcodec libx264 /content/output_with_detection.mp4

# Download the output video
from google.colab import files
files.download('/content/output_with_detection.mp4')


In [ ]:
def process_new_video(video_path, output_path='/content/detected_new_video_output.avi'):
    capture = cv2.VideoCapture(video_path)
    assert capture.isOpened()

    im_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    im_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, 20, (im_width, im_height))

    while True:
        ret, frame = capture.read()
        if not ret:
            break

        detected_frame = detect_objects(frame)
        out.write(np.uint8(detected_frame))

    capture.release()
    out.release()
    print(f"Output video saved as {output_path}")


In [ ]:
# Test with a new video
new_video_path = '/content/drive/MyDrive/path_to_your_new_video.mp4'  # Update with the path of the new video file
process_new_video(new_video_path)
